GPT знает больше чем любой человек, но значит, ли что она умнее?
Попробуем сыграть с ней в игру.

Давай сыграем в игру. Каждый пишет слова по очереди. Сначала я одно, потом ты, потом снова я и так далее. Задача чтобы получался связанный текст. Выигрывает тот, кто первым после 10 слов (в сумме те твои 5 и мои 5) напишет слово рецепт (в любой форме). Если кто-то из нас написав слово которое делает текст несвязным автоматически проигрывает. Правила понятны? Можем начинать?

In [53]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from dotenv import load_dotenv
load_dotenv(override=True)

MODEL_GPT3 = "gpt-3.5-turbo"
MODEL_GPT4 = "gpt-4-1106-preview"
MODEL = MODEL_GPT4
MODEL_TEMP = 0

def init_chain(template):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", template),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{input}"),
        ]
    )

    memory = ConversationBufferMemory(return_messages=True)
    model = ChatOpenAI(model=MODEL, temperature=MODEL_TEMP)
    chain = (
        RunnablePassthrough.assign(
            history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
        )
        | prompt
        | model
    )
    return chain, memory

def send_message(message, memory, chain):
    memory.load_memory_variables({})
    inputs = {"input": message}
    response = chain.invoke(inputs)
    memory.save_context(inputs, {"output": response.content})
    return response.content

In [55]:
template = "Мы играем в игру. Каждый пишет слова по очереди. Сначала я одно, потом ты, потом снова я и так далее. Задача чтобы получался связанный текст. Выигрывает тот, кто первым после 10 слов (в сумме те твои 5 и мои 5) напишет слово рецепт (в любой форме). Если кто-то из нас написав слово которое делает текст несвязным автоматически проигрывает."
chain, memory  = init_chain(template)
memory.clear()
try:
    while True:
        message = input("Введите слово: ")
        if message=="stop":
            break
        else:
            print(f"Вы> {message}")
            print(f"Я> {send_message(message, memory, chain)}")
except KeyboardInterrupt:
    pass

print("Спасибо за игру")

/Users/serafim/Lab/GPT_Finish_The_Sentence/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


Вы> Море
Я> волнуется
Вы> раз.
Я> Ветер
Вы> волнуется
Я> тоже.
Вы> Я
Я> люблю
Вы> пироги
Я> смотреть
Спасибо за игру


In [56]:
msg_to_validate = " ".join([msg.content for msg in memory.load_memory_variables({})['history']][0:-1])
msg_to_validate

'Море волнуется раз. Ветер волнуется тоже. Я люблю пироги'

In [57]:
validation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты эксперт в лингвистики и помогаешь мне оценить насколько связным получились предложения само по себе и если несколько предложений, то насколько они связанны так же между собой. Пожалуйста оцени текст по шкале от 1 до 10, где 1 - текст не связный, 10 - текст связный."),
        ("human", "{input}"),
    ]
)

validation_model = ChatOpenAI(model=MODEL, temperature=MODEL_TEMP)
validation_chain = (
     validation_prompt
    | validation_model
)

/Users/serafim/Lab/GPT_Finish_The_Sentence/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [58]:
print(validation_chain.invoke({"input": msg_to_validate}).content)

Текст состоит из трех предложений, которые кажутся отдельными мыслями без явной связи между ними. Первые два предложения связаны темой природных явлений (море и ветер), но третье предложение о пирогах не связано с предыдущими и кажется неуместным в данном контексте.

На основе этого я бы оценил связность текста на 3 из 10, так как есть некоторая связь между первыми двумя предложениями, но общая связность нарушается внезапным переходом к совершенно другой теме.


In [59]:
sentances = msg_to_validate.split(".")

In [60]:
for sentance in sentances:
    print(sentance+":\n")
    print(validation_chain.invoke({"input": sentance}).content)
    print("\n\n")

Море волнуется раз:

Это предложение само по себе является фрагментом известной детской считалочки или песенки и без дополнительного контекста оно не несет много информации. Однако, оно может быть воспринято как полное, если рассматривать его как поэтическое или философское утверждение. В таком случае, оно может вызвать образ моря и его волнения, что делает его связным в рамках литературного выражения.

Если оценивать это предложение как часть более широкого текста, то для полной оценки связности необходимо увидеть остальные предложения. В изолированном виде, я бы оценил это предложение на 5 по шкале связности, так как оно может быть интерпретировано разными способами в зависимости от контекста.



 Ветер волнуется тоже:

Это предложение само по себе не очень информативно и может быть интерпретировано по-разному в зависимости от контекста. Оно может быть частью стихотворения, песни или философского высказывания. Однако, без дополнительного контекста, его связность оценить сложно. Если 

In [ ]:
chain2, memory2 = init_chain(template)

In [61]:
def debate(template):
    chain, memory = init_chain(template)
    chain2, memory2 = init_chain(template)
    word = input("Первое слово: ")
    print(f"Я> {word}")
    i = 1
    try:
        while True:
            word = send_message(word, memory, chain)
            print(f"AI1[{i}]> {word}")
            word = send_message(word, memory2, chain2)
            print(f"AI2[{i+1}]> {word}")
            i += 2
            message = input("Продолжаем (y/n)?: ")
            if message!="y":
                break
    except KeyboardInterrupt:
        pass

    print("Спасибо за игру")

In [62]:
debate(template)

/Users/serafim/Lab/GPT_Finish_The_Sentence/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/Users/serafim/Lab/GPT_Finish_The_Sentence/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


Я> Море
AI1[1]> волнуется
AI2[2]> озеро
AI1[3]> спокойно
AI2[4]> отражая
AI1[5]> лунный
AI2[6]> свет
AI1[7]> ночью
AI2[8]> рецепт
Спасибо за игру


In [63]:
debate("Мы играем в игру. Каждый пишет слова по очереди. Сначала я одно, потом ты, потом снова я и так далее. Задача чтобы получался связанный текст. Выигрывает тот, кто первым после 10 слов (в сумме те твои 5 и мои 5) напишет слово рецепт (в любой форме). Если кто-то из нас написав слово которое делает текст несвязным автоматически проигрывает. Вместе со словами можно использовать знаки припинания.")

/Users/serafim/Lab/GPT_Finish_The_Sentence/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/Users/serafim/Lab/GPT_Finish_The_Sentence/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


Я> Море
AI1[1]> волнуется
AI2[2]> море
AI1[3]> разбивается
AI2[4]> о
AI1[5]> берег
AI2[6]> и
AI1[7]> песок
AI2[8]> засыпает
AI1[9]> берега.
AI2[10]> Вода
AI1[11]> отступает,
AI2[12]> оставляя
AI1[13]> ракушки.
AI2[14]> Рецепт
AI1[15]> Поздравляю, вы выиграли игру!
AI2[16]> Спасибо за игру! Это было интересное испытание нашей способности создавать связный текст. Если захотите сыграть еще раз или обсудить что-то другое, я всегда здесь, чтобы помочь!
Спасибо за игру


In [64]:
debate('Мы играем в игру. Каждый пишет слова по очереди. Сначала я одно, потом ты, потом снова я и так далее. Задача чтобы получался связанный текст. Чтобы выиграть, нужно использовать слово "рецепт" в тексте таким образом, чтобы после его использования когда предложение заканчивается, и при этом никто не должен использовать слово "ртуть" до завершения этого предложения. Если же слово "ртуть" будет использовано до завершения предложения с "рецепт", то это не считается победой, и игра продолжается. Если кто-то из нас написав слово которое делает текст несвязным автоматически проигрывает. Вместе со словами можно использовать знаки припинания.')

/Users/serafim/Lab/GPT_Finish_The_Sentence/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
/Users/serafim/Lab/GPT_Finish_The_Sentence/myenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


Я> Море
AI1[1]> было
AI2[2]> утро
AI1[3]> и
AI2[4]> солнце
AI1[5]> нежно
AI2[6]> гладило
AI1[7]> поверхность
AI2[8]> озера
AI1[9]> .
AI2[10]> Воздух
AI1[11]> был
AI2[12]> наполнен
AI1[13]> ароматами
AI2[14]> цветущих
AI1[15]> растений
AI2[16]> и
AI1[17]> свежеиспеченного
AI2[18]> хлеба
AI1[19]> .
AI2[20]> На
AI1[21]> кухне
AI2[22]> бабушка
AI1[23]> тщательно
AI2[24]> выбирала
AI1[25]> специи
AI2[26]> для
AI1[27]> секретного
AI2[28]> рецепта
AI1[29]> .
AI2[30]> Победа!
AI1[31]> Поздравляю! Вы использовали слово "рецепт" в тексте, и предложение было завершено без использования слова "ртуть". Вы выиграли в эту игру.
AI2[32]> Спасибо за игру! Это было интересное испытание. Если захотите сыграть снова или обсудить что-то еще, я всегда здесь.
Спасибо за игру


Ну вот как-то так... не хочет работать. Давайте искать как улучшить.